# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [23]:
# In[1]:


import scipy.signal as scs

import glob

import numpy as np
import scipy as sp
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)



def RunPulseRateAlgorithm(data_fl, ref_fl):
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    #set sampling frequency to 125 Hz
    fs=125
    
    # Convert the frequency from Beat Per Minute to Hz
    # 40 BPM and 240 BPM is the recommended lower limit and higher limit for heartbeat signal, respectively.
    lower_freq_limit,higher_freq_limit =(40/60,
                                          240/60)
    freq_limits = lower_freq_limit,higher_freq_limit
    
    ### use bandpass filter (BPF) to filter out the signal in order to focus on the typical heartbeat frequency range
    ppg=BPF(ppg, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accx=BPF(accx, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accy=BPF(accy, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accz=BPF(accz, lower_freq_limit,higher_freq_limit, sampling_freq = fs)

    # Compute pulse rate estimates and estimation confidence.
    #########################################################    
    pr, confidence = peakdetection(ppg, accx, accy, accz)  
    #########################################################  
    

    # Load the pulse rate reference/ground truths array
    pr_reference=sp.io.loadmat(ref_fl)['BPM0'].reshape((1,-1))[0] 
    
    # Calculate the error, with considering the difference between reference vs produced pulse rate array length 
    if len(pr_reference) < len(pr):
        errors=abs(pr[:len(pr_reference)]-pr_reference)
    elif len(pr_reference) > len(pr):
        errors=abs(pr_reference[:len(pr)]-pr)
    else :
        errors=abs(pr[:len(pr_reference)]-pr_reference)
        
    confidence=np.array(confidence[:len(pr)]) 

    # standard_output will return errors and confidence only, per project instruction (default)
    final_output =  errors, confidence
        
    return  final_output


# In[4]:


def BPF(signal, lower_freq_limit, higher_freq_limit, sampling_freq):
    
    b, a = scs.butter(3, (lower_freq_limit,higher_freq_limit ), btype='bandpass', fs=sampling_freq)
    filtered_signal=  scs.filtfilt(b, a, signal)
    
    return filtered_signal


# In[31]:


wl = 1000

def peakdetection(ppg_st1, accx_st1, accy_st1, accz_st1):
    
    # pr is the produced estimation of pulse rate
    pr=[]
    

    confidence=[]   
    
    fs = 125
    
    window_length=wl
    
    # Considering Nyquist theorem, windows shift is set to twice the sampling frequency  
    window_shift=2*fs 
    
    # Sampling the time-domain signal for each window shift length
    for i in range(0, len(ppg_st1) - window_length, window_shift):
        ppg_sample=ppg_st1[i: i + window_length]
        accx_sample=accx_st1[i: i + window_length]
        accy_sample=accy_st1[i: i + window_length]
        accz_sample=accz_st1[i: i + window_length]
        
        # Set the frequency range to be used in the fourier domain
        freqs = np.fft.rfftfreq(window_length, 1/fs)
        # Convert the PPG signal to the frequency domain using 1D discrete Fourier Transform (with real input)
        p_fourier = abs(np.fft.rfft(ppg_sample))
        
        # Convert the 3 accelerometer signals to the frequency domain using 1D discrete Fourier Transform (with real input)
        x_fourier = abs(np.fft.rfft(accx_sample))
        y_fourier = abs(np.fft.rfft(accy_sample))
        z_fourier = abs(np.fft.rfft(accz_sample))
        
        # Accumulate the 3 accelerometer signals into a combined accelerometer signal
        xyz_fourier=np.square(x_fourier)+np.square(y_fourier)+np.square(z_fourier)
        xyz_fourier=np.sqrt(xyz_fourier)

        # Detect any peaks in the PPG signal,
        p_fourier_peaks_xdim,p_fourier_peaks_ydim = sp.signal.find_peaks(p_fourier,height=400)
        # Define the x-coordinate of these peaks (if any),
        p_fourier_peaks_xdim=freqs[p_fourier_peaks_xdim]
        # Define the y-coordinate/height of these peaks (if any),
        p_fourier_peaks_ydim=p_fourier_peaks_ydim['peak_heights']

        # remove any unwanted peaks (which is present outside the heartbeat frequency range) in PPG signal
        p_fourier_freq_range=(p_fourier_peaks_xdim>=40/60)&(p_fourier_peaks_xdim<=240/60)
        p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim[p_fourier_freq_range]
        p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim[p_fourier_freq_range]   
        
        # Detect any peaks in the Combined Accelerometer(CA) signal
        xyz_fourier_peaks_xdim,xyz_fourier_peaks_ydim = sp.signal.find_peaks(xyz_fourier,height=5)
        # Define the x-coordinate of these peaks (if any),
        xyz_fourier_peaks_xdim=freqs[xyz_fourier_peaks_xdim]
        # Define the y-coordinate/height of these peaks (if any),
        xyz_fourier_peaks_ydim=xyz_fourier_peaks_ydim['peak_heights']

        # remove any unwanted peaks (which is present outside the heartbeat frequency range) in CA signal
        xyz_fourier_freq_range=(xyz_fourier_peaks_xdim>=40/60)&(xyz_fourier_peaks_xdim<=240/60)
        xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim[xyz_fourier_freq_range]
        xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim[xyz_fourier_freq_range]

        # When a peak within PPG frequencies is detected, compare the top 4 peaks with top 4 peaks from CA frequencies,
        # and remove any shared peaks in order to avoid artifact due to subject movement
        if len(p_fourier_peaks_xdim_filtered)>0:
            
            # Sort the coordinates based on the highest peaks
            p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim_filtered[np.argsort(-p_fourier_peaks_ydim_filtered)]
            p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim_filtered[np.argsort(-p_fourier_peaks_ydim_filtered)] 
            
            # If more than 4 peaks are detected, only process the highest 4 peaks
            if len(p_fourier_peaks_xdim_filtered)>4:
                p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim_filtered[:4]
                p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim_filtered[:4]
            
            # If one or more peaks within CA frequencies is detected,....
            if len(xyz_fourier_peaks_xdim_filtered)>0: 
                xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim_filtered[np.argsort(-xyz_fourier_peaks_ydim_filtered)]
                xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim_filtered[np.argsort(-xyz_fourier_peaks_ydim_filtered)]
            
            # .... process only the highest 4 peaks
            if len(xyz_fourier_peaks_xdim_filtered)>4: 
                xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim_filtered[:4]
                xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim_filtered[:4]
            
            # Check for unique frequencies (detected frequency peaks in PPG signal that does not exist in CA signal)
            unique_f=[x_coord for x_coord in p_fourier_peaks_xdim_filtered if not x_coord  in xyz_fourier_peaks_xdim_filtered]
            
            # Choose only the first detected unique frequency (if there is any) OR simply choose 
            # the first shared peak (PPG-AC) (if unique PPG frequency does not exist) as the pulse rate in Hz
            if len(unique_f)>0:
                pr_f=unique_f[0]
            else:
                pr_f=p_fourier_peaks_xdim_filtered[0]

        # If there is no PPG signal peak detected in the first place, use the strongest frequency in PPG signal as the pulse rate in Hz
        else:
            freqs_range_nopeak=(freqs>=40/60)&(freqs<=240/60)
            freqs_filtered_nopeak=freqs[freqs_range_nopeak]
            p_fourier_filtered_nopeak=p_fourier[freqs_range_nopeak]
            pr_f=freqs_filtered[np.argsort(-p_fourier_filtered)][0]
        
        
        # Convert the pulse rate in Hz to BPM
        pr_bpm=pr_f*60
        
        # Since pulse rate reading can be very fluctuative and unstable, some sort stabilizing algorithm is introduced
        # by updating pulse rate if the calculated pulse rate value is different enough (more than 10 percent difference) 
        # compared to the previous pulse rate reading. ALSO, the updated value is set to ONLY either 10 percent 
        # lower or higher depending on the calculated pulse rate vs recorded pulse rate 
        # Credit to github.com/neilzhang
        if len(pr)>0:
            if pr_bpm>1.1*pr[-1]:
                pr_bpm=1.1*pr[-1]
            if pr_bpm<pr[-1]/1.1:
                pr_bpm=pr[-1]/1.1
                
        # Adding a value to the pulse rate array, it is either a same value with the last reading, 
        # 10 percent higher than the last reading, or 10 percent lower than last reading.
        pr.append(pr_bpm)
        
        # The confidence is quantified as the proportion between highlighted PPG frequency 
        # (which is having slightest difference with detected pulse rate frequency magnitude),
        # compared against total magnitude of frequencies detected
        diff_p_fourier_peaks_xdim=abs(p_fourier_peaks_xdim-pr_bpm/60)
        confidence.append(p_fourier_peaks_ydim[diff_p_fourier_peaks_xdim.argmin()]/sum(p_fourier_peaks_ydim)) 
        
    return pr, confidence


Evaluate()

8.9707247890142803